In [ ]:
# merge all json data in labelstudio_data/
python read_and_merge_data.py   

In [ ]:
# split merged data (read_and_merge_data.py output) into train/test/dev
python label_studio.py  --options ./labelstudio_data/label.txt

In [ ]:
# train
python run_train.py  \
    --device gpu \
    --logging_steps 5 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 1000 \
    --model_name_or_path utc-base \
    --output_dir ./checkpoint/model_best \
    --dataset_path ./data/ \
    --max_seq_length 700  \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --num_train_epochs 20 \
    --learning_rate 3e-6 \
    --do_train \
    --do_eval \
    --do_export \
    --export_model_dir ./checkpoint/model_best \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model macro_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1 \
    --save_plm